In [2]:
import kagglehub
import os


# Download latest version
path = kagglehub.dataset_download("s076923/pytorch-transformer")
print("Path to dataset files:", path)

/Users/carki/Desktop/Dev/codeit/codeit/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 916M/916M [01:33<00:00, 10.3MB/s] 

Extracting files...


Path to dataset files: /Users/carki/.cache/kagglehub/datasets/s076923/pytorch-transformer/versions/4


In [4]:
# 먼저, 압축 파일(pytorch-transformer.zip)을 지정된 경로(현재 디렉토리)로 압축 해제합니다.
import shutil

# shutil.move(path, '/Users/carki/Desktop/Dev/codeit/common/data')


In [5]:
# 작업 디렉토리를 변경합니다.
import os
os.chdir("/Users/carki/Desktop/Dev/codeit/common/data/4/datasets")  # 작업할 데이터셋 폴더로 이동 (실행 환경에 따라 경로가 달라질 수 있음)

```json
{
	"info": {
		"year": 2021,
		"version": "1.0",
		"description": "For object detection",
		"date_created": "2021"
	},
	"images": [
		{
			"date_captured": "2021",
			"file_name": "000000000001.jpg",
			"id": 1,
			"height": 480,
			"width": 640
		},
		{
			"date_captured": "2021",
			"file_name": "000000000002.jpg",
			"id": 2,
			"height": 426,
			"width": 640
		},
		{
			"date_captured": "2021",
			"file_name": "000000000003.jpg",
			"id": 3,
			"height": 428,
			"width": 640
		},
		{
			"date_captured": "2021",
			"file_name": "000000000004.jpg",
			"id": 4,
			"height": 425,
			"width": 640
		},
		{
			"date_captured": "2021",
			"file_name": "000000000005.jpg",
			"id": 5,
			"height": 640,
			"width": 481
		}
	],
	"licenses": [
		{
			"id": 1,
			"name": "GNU General Public License v3.0",
			"url": "https://github.com/zhiqwang/yolov5-rt-stack/blob/master/LICENSE"
		}
	],
	"type": "instances",
	"annotations": [
		{
			"segmentation": [
				[
					1.0799999999999272,
					187.69008000000002,
					612.66976,
					187.69008000000002,
					612.66976,
					473.53008000000005,
					1.0799999999999272,
					473.53008000000005
				]
			],
			"area": 174816.81699840003,
			"iscrowd": 0,
			"image_id": 1,
			"bbox": [
				1.0799999999999272,
				187.69008000000002,
				611.5897600000001,
				285.84000000000003
			],
			"category_id": 19,
			"id": 1
		},
		{
			"segmentation": [
				[
					311.73024,
					4.310159999999996,
					631.0102400000001,
					4.310159999999996,
					631.0102400000001,
					232.99032,
					311.73024,
					232.99032
				]
			],
			"area": 73013.00148480001,
			"iscrowd": 0,
			"image_id": 1,
			"bbox": [
				311.73024,
				4.310159999999996,
				319.28000000000003,
				228.68016
			],
			"category_id": 50,
			"id": 2
		},
        ],
	"categories": [
		{
			"id": 1,
			"name": "0",
			"supercategory": "0"
		},
		{
			"id": 2,
			"name": "1",
			"supercategory": "1"
		},
		{
			"id": 3,
			"name": "2",
			"supercategory": "2"
		},
    ]
}
```

In [22]:
import os              # 파일 및 디렉토리 경로를 다루기 위한 표준 라이브러리
import json            # JSON 파일을 읽고 쓰기 위한 표준 라이브러리
import torch           # 딥러닝 라이브러리 PyTorch (텐서 연산 등)
from PIL import Image  # 이미지를 다루기 위한 Pillow 라이브러리
from torch.utils.data import Dataset  # PyTorch의 Dataset 클래스를 상속받기 위한 모듈

# COCO 데이터셋의 JSON 파일을 직접 파싱하기 위한 사용자 정의 클래스
class CustomCOCO:
    def __init__(self, annotation_file):

        # JSON 파일을 읽어서 데이터를 불러온다.
        with open(annotation_file, 'r') as f:
            self.data = json.load(f)

        # 이미지 정보 ---> 이미지 'id'를 key로 하는 dict로 저장
        self.images = {img["id"]: img for img in self.data.get("images", [])}

        # Annotation 정보를 이미지 'id'로 그룹화
        self.annotation = {}

        for ann in self.data.get('annotations', []):
            img_id = ann['image_id']

            if img_id not in self.annotation:
                self.annotation[img_id] = []
            
            self.annotation[img_id].append(ann)

        # 카테고리 정보를 cat 'id'를 키로 하는 dict로 저장
        self.cats = { cat['id']: cat for cat in self.data.get('categories', []) }
        # self.cats = {}
        # for cat in self.data.get('categories', []):
        #     self.cats[cat['id']] = cat

# 이미지 id들을 통해서 이미지 정보들을 불러온다
def loadImgs(self, ids):
    return [self.images[i] for i in ids if i in self.images]

# 이미지 id를 통해서 Annotation id를 불러온다
def getAnnIds(self, imgIds):
    ann_ids = []
    for img_id in imgIds:
        if img_id in self.annotation:
            ann_ids.extend([ann['id'] for ann in self.annotation[img_id]])

    return ann_ids

# Annotation id를 이용해서 실제 annotation 정보를 불러온다.
def loadAnns(self, annIds):
    anns = []
    for ann in self.data.get('annotations', []):
        if ann['id'] in annIds:
            anns.append(ann)

    return anns

# Pytorch Dataset 클래스를 상속받아서 coco 데이터셋 클래스 생성

In [23]:
class cocoDataset(Dataset):
    def __init__(self, root, train, transform=None):
        super().__init__()
        # root : 데이터셋 위치(최상위)
        # train : 학습 데이터(True), 검증데이터(False)

        # dir = 'train' if train else 'val'
        if train:
            dir = 'train'
        else:
            dir = 'val'

        # anno file의 위치를 지정
        annotation_file = os.path.join(root, 'annotations', f'{dir}_annotations.json')

        # CustomCOCO 클래스 지정
        self.coco = CustomCOCO(annotation_file=annotation_file)

        # image file 위치
        self.image_path = os.path.join(root, dir)

        self.transform = transform

        # coco 데이터셋의 카테고리 정보를 저장
        # 0번은 배경정보로 지정
        self.categories = { 0: 'background' }
        
        for cat_id, cat in self.coco.cats.items():
            self.categories[cat_id] = cat['name']

        # 이미지와 어노테이션 정보를 로드 ---> data 리스트에 저장
        self.data = self._load_data()

    def _load_data(self):
        data = []

        for _id, img_info in self.coco.images.items():
            # 이미지 파일 가져오기
            file_name = img_info['file_name']

            # 이미지 파일의 전체 경로 가져오기
            image_path = os.path.join(self.image_path, file_name)

            # PIL를 사용해서 이미지를 로드 ---> RGB
            image = Image.open(image_path).convert('RGB')

            boxes = []
            labels = []
            anns = self.coco.annotation.get(_id, [])

            for ann in anns:
                x, y, w, h = ann['bbox']
                boxes.append([x, y, x+w, y+h])
                labels.append(ann['category_id'])

            # target 값들을 dict 형태로 만들어서 전달
            taget = {
                'image_id': torch.LongTensor([_id]),
                'boxes': torch.FloatTensor(boxes),
                'labels': torch.LongTensor(labels)
            }

            data.append((image, taget))
        
        return data

    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, index):
        image, target = self.data[index]
        
        if self.transform:
            images = self.transform(image)
        
        return image, target

# Dataloader 생성

In [35]:
# torchvision 라이브러리에서 이미지 전처리 도구와 DataLoader를 임포트합니다.
from torchvision import transforms
from torch.utils.data import DataLoader

# 배치 데이터를 생성할 때, 각 배치마다 데이터를 튜플 형태로 묶어주는 함수
# coco 데이터 셋은 이미지 내에 여러 객체 정보가 담길 수 있으므로, 데이터의 길이가 다를 수 있음.
def collator(batch):
    return tuple(zip(*batch))

transform = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(dtype=torch.float)
    ]
)

train_dataset = cocoDataset(root='/Users/carki/Desktop/Dev/codeit/common/data/4/datasets/coco', train=True, transform=transform)
test_dataset = cocoDataset(root='/Users/carki/Desktop/Dev/codeit/common/data/4/datasets/coco', train=False, transform=transform)

train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=4,
                              shuffle=True,
                              drop_last=True,
                              collate_fn=collator)
test_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=1,
                              shuffle=False,
                              drop_last=True,
                              collate_fn=collator)

In [36]:
train_dataset[101]

(<PIL.Image.Image image mode=RGB size=640x471>,
 {'image_id': tensor([374990]),
  'boxes': tensor([[242.1200, 182.9700, 427.7500, 399.7300]]),
  'labels': tensor([2])})

# Custom Collator가 필요한 이유

아래에 기본 collate 함수의 출력과 custom collate를 적용한 최종 출력 형태를 요약했습니다.

- **기본 collate 함수 (원래 형태):**
  - **출력:**  
    - **이미지 텐서:** 모든 이미지가 동일한 크기라면, 자동으로 스택되어 하나의 텐서로 만들어집니다.  
      예: `(batch_size, channels, height, width)`
    - **타겟(라벨) 텐서/딕셔너리:** 동일한 방식으로 스택 또는 텐서 형태로 구성됩니다.
  - **문제점:**  
    - 이미지나 타겟의 크기가 서로 다르면 스택하는 과정에서 에러가 발생합니다.

- **Custom collate 적용 후 (최종 원하는 형태):**
  - **출력:**  
    - **이미지 리스트:** 각 이미지가 개별적으로 리스트에 담깁니다.  
      예: `(image1, image2, ...)`
    - **타겟 리스트:** 각 이미지에 해당하는 타겟이 개별적으로 리스트에 담깁니다.  
      예: `(target1, target2, ...)`
  - **형태 요약:**  
    - 최종 배치의 출력은 **((이미지1, 이미지2, ...), (target1, target2, ...))** 형태로 구성됩니다.
  - **장점:**  
    - 이미지나 타겟이 서로 다른 크기여도 그대로 유지할 수 있어, 모델 입력 전에 적절한 전처리(예: 패딩)를 적용하거나, 개별적으로 처리할 수 있습니다.

이렇게 custom collate를 사용하면 서로 다른 크기의 데이터를 안전하게 배치로 묶어 모델에 입력할 수 있습니다.

# 2개의 샘플 이미지 (각각 다른 크기의 tensor)

In [26]:
import torch
from torch.utils.data import dataloader

image1 = torch.randn(3, 300, 300)
target1 = {
    'boxes': torch.tensor([
        [50, 50, 200, 200]], 
        dtype=torch.float32),
    'labels': torch.tensor([1])
}

image2 = torch.randn(3, 400, 400)
target2 = {
    'boxes': torch.tensor(
        [
            [10, 50, 200, 200], 
            [10, 60, 200, 200]
        ], 
        dtype=torch.float32
    ),
    'labels': torch.tensor([1])
}

sample_data = [(image1, target1), (image2, target2)]
loader_without_collator = DataLoader(dataset=sample_data,
                                     batch_size=2)

# 에러발생:  stack expects each tensor to be equal size, but got [3, 300, 300] at entry 0 and [3, 400, 400] at entry 1
try:
    for batch in loader_without_collator:
        images, target = batch
        print('Batch img shape: ', images.shape)
except Exception as e:
    print('에러발생: ', e)

def collator(batch):
    return tuple(zip(*batch))

loader_with_collator = DataLoader(dataset=sample_data,
                                  batch_size=2, 
                                  collate_fn=collator)

for batch in loader_with_collator:
    images, targets = batch
    print('batch 이미지 수: ', len(images))
    print('1st 이미지 크기: ', images[0].shape)
    print('2st 이미지 크기: ', images[1].shape)
    print('1st target: ', targets[0])
    print('2st target: ', targets[1])

에러발생:  stack expects each tensor to be equal size, but got [3, 300, 300] at entry 0 and [3, 400, 400] at entry 1
batch 이미지 수:  2
1st 이미지 크기:  torch.Size([3, 300, 300])
2st 이미지 크기:  torch.Size([3, 400, 400])
1st target:  {'boxes': tensor([[ 50.,  50., 200., 200.]]), 'labels': tensor([1])}
2st target:  {'boxes': tensor([[ 10.,  50., 200., 200.],
        [ 10.,  60., 200., 200.]]), 'labels': tensor([1])}


In [27]:
import torch

from torchvision import models
from torchvision import ops
from torchvision.models.detection import rpn
from torchvision.models.detection import FasterRCNN

# 사전 학습된 VGG16 모델의 Feature 부분만 backbone으로 사용
backbone = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1').features

# backbone의 출력 채널을 512로 설정
backbone.out_channels = 512

anchor_generator = rpn.AnchorGenerator(
    sizes=((32, 64, 128, 256, 512),), # 앵커의 크기 지정
    aspect_ratios=((0.5, 1.0, 2.0))   # 앵커의 가로세로 비율 지정
)

# ROI Pooling : 여러 스케일의 특징맵에서 ROI를 고정 크기로 변환
roi_pooler = ops.MultiScaleRoIAlign(
    featmap_names=['0'],    # 사용할 특징맵의 이름
    output_size=(7, 7),     # 최종적으로 나갈 사이즈
    sampling_ratio=2,       # 한 그리드 셀에서 몇개의 샘플을 활용해 데이터를 가져올거냐?, 샘플이 많을수록 원본에 가깝지만 연산량이 많아짐
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FasterRCNN(
    backbone=backbone,
    num_classes=3,   # 강아지, 고양이, 배경
    rpn_anchor_generator=anchor_generator,
    box_roi_pool=roi_pooler
).to(device)  # 모델을 선택한 디바이스로 이동

In [28]:
model

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(in

In [29]:
# 모델 학습을 위한 최적화 기법 설정 (SGD 사용)
from torch import optim

params = [p for p in model.parameters() if p.requires_grad]  # 학습 가능한 파라미터들만 선택
optimizer = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
# 학습률 스케줄러: 일정 에폭마다 학습률 감소
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
from tqdm import tqdm  # 진행 상황을 표시하기 위한 라이브러리

# 학습 루프 (총 5 에폭 동안 학습)
num_epochs = 5
for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 전환
    total_loss = 0.0

    # tqdm 진행바를 사용하여 학습 진행 상황 표시
    train_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs} - Training")
    for images, targets in train_bar:
        # 이미지와 타겟을 디바이스로 이동
        # images.to(device)로는 처리가 안됨
        
        # 데이터가 텐서 형태로 있지 않기 때문에
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # 모델에 입력하여 손실(loss) 계산
        # model에서 output이 loss dict형태로 보냄
        loss_dict = model(images, targets)
        # loss_dict에서 값(val)만 추출해서 모두 sum
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()  # 기울기 초기화
        losses.backward()      # 역전파 수행
        optimizer.step()       # 파라미터 업데이트

        total_loss += losses.item()
        train_bar.set_postfix(loss=f"{losses.item():.3f}")

    lr_scheduler.step()  # 에폭이 끝난 후 학습률 업데이트
    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Epoch: {epoch+1:2d}, Avg Train Loss: {avg_train_loss:.3f}")

    # 사용하지 않는 GPU 메모리 해제 (메모리 최적화)
    torch.cuda.empty_cache()

Epoch 1/5 - Training:   0%|          | 0/607 [00:00<?, ?it/s]


AttributeError: 'Image' object has no attribute 'to'

In [ ]:
### 테스트 및 시각화 관련 코드 ###

import numpy as np
from matplotlib import pyplot as plt
from torchvision.transforms.functional import to_pil_image

# 바운딩 박스를 그림으로 표시하는 함수
def draw_bbox(ax, box, text, color):
    """
    바운딩 박스를 이미지에 그려주는 함수.
    :param ax: matplotlib Axes 객체
    :param box: 바운딩 박스 좌표 (x_min, y_min, x_max, y_max)
    :param text: 바운딩 박스 위에 표시할 텍스트
    :param color: 바운딩 박스와 텍스트의 색상
    """
    ax.add_patch(
        plt.Rectangle(
            xy=(box[0], box[1]),
            width=box[2] - box[0],
            height=box[3] - box[1],
            fill=False,
            edgecolor=color,
            linewidth=2,
        )
    )
    ax.annotate(
        text=text,
        xy=(box[0] - 5, box[1] - 5),
        color=color,
        weight="bold",
        fontsize=13,
    )

num_vis = 0
threshold = 0.5  # 예측 점수 임계값: 이 값 이상인 예측만 시각화에 사용
categories = test_dataset.categories  # COCO 데이터셋의 카테고리 정보 사용
with torch.no_grad():  # 평가 시에는 기울기 계산을 하지 않습니다.
    model.eval()       # 모델을 평가 모드로 전환
    for images, targets in test_dataloader:
        images = [image.to(device) for image in images]
        outputs = model(images)  # 모델로부터 예측 결과 생성

        # 예측 결과에서 바운딩 박스, 라벨, 점수를 CPU의 numpy 배열로 변환
        boxes = outputs[0]["boxes"].to("cpu").numpy()
        labels = outputs[0]["labels"].to("cpu").numpy()
        scores = outputs[0]["scores"].to("cpu").numpy()

        # 임계값 이상의 예측만 선택
        boxes = boxes[scores >= threshold].astype(np.int32)
        labels = labels[scores >= threshold]
        scores = scores[scores >= threshold]

        # 시각화를 위한 플롯 설정 (8x8 크기의 그림)
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(1, 1, 1)
        plt.imshow(to_pil_image(images[0]))  # 첫 번째 이미지를 PIL 이미지로 변환하여 출력

        # 예측 결과(빨간색 박스) 시각화
        for box, label, score in zip(boxes, labels, scores):
            draw_bbox(ax, box, f"{categories[label]} - {score:.4f}", "red")

        # 정답 어노테이션(파란색 박스) 시각화
        tboxes = targets[0]["boxes"].numpy()
        tlabels = targets[0]["labels"].numpy()
        for box, label in zip(tboxes, tlabels):
            draw_bbox(ax, box, f"{categories[label]}", "blue")

        plt.show()  # 시각화 결과 출력
        num_vis += 1
        if num_vis == 5:
            break
